# Day 4 - Lab 1: Automated Testing & Quality Assurance

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 4! An application without tests is an application that is broken by design. Today, we focus on quality assurance. You will act as a QA Engineer, using an AI co-pilot to build a robust test suite for the API you created yesterday. This is a critical step to ensure our application is reliable and ready for production.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the source code for our main application from `app/main.py`. Providing the full code as context is essential for the LLM to generate accurate and relevant tests.

**Model Selection:**
For generating tests, models with strong code understanding and logical reasoning are best. `gpt-4.1`, `o3`, `codex-mini`, and `gemini-2.5-pro` are all excellent choices for this task.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated test files.
- `clean_llm_output()`: To clean up the generated Python code.

In [11]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

# Load the application code from Day 3 to provide context for test generation
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

2025-09-25 11:39:57,441 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4.1 latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Task:** Generate basic `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

**Instructions:**
1.  Create a prompt that asks the LLM to act as a QA Engineer.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to generate a `pytest` test function for the `POST /users/` endpoint, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body).
4.  Generate another test for the `GET /users/` endpoint.
5.  Save the generated tests into a file named `tests/test_main_simple.py`.

**Expected Quality:** A Python script containing valid `pytest` functions that test the basic, successful operation of your API.

In [12]:
# Prompt for LLM to generate happy path tests for the API
happy_path_tests_prompt = f"""
You are a QA Engineer. Given the following FastAPI application code, write a Python script using pytest to test the basic, successful operation of the API's CRUD endpoints.

Requirements:
- Write a test function for the POST /users/ endpoint. The test should create a user, assert that the response status code is 201 (or 200), and verify that the response body contains the expected user data.
- Write a test function for the GET /users/ endpoint. The test should retrieve the list of users and assert that the response status code is 200 and the response body contains the expected data.
- Use pytest conventions and best practices.
- The tests should be self-contained and runnable as a script.
- Do not include edge cases or error scenarios.

Application code context:
""" + app_code + """

(no markdown fences)
"""

print("--- Generating Happy Path Tests ---")
if app_code:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_main_simple.py", overwrite=True)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Happy Path Tests ---
# test_users_api.py

import pytest
from fastapi.testclient import TestClient
from datetime import datetime
from main import app  # Assumes the FastAPI app code is in main.py

client = TestClient(app)

@pytest.fixture
def user_payload():
    return {
        "first_name": "Alice",
        "last_name": "Smith",
        "email": f"alice{datetime.utcnow().timestamp()}@example.com",  # ensure unique email
        "password_hash": "s3cr3t",
        "role_id": 1,
        "manager_id": None,
        "team_id": None,
        "start_date": "2023-01-01T09:00:00",
        "gamification_points": 10,
        "profile_picture_url": "https://example.com/alice.jpg"
    }

def test_create_user(user_payload):
    response = client.post("/users/", json=user_payload)
    assert response.status_code in (200, 201)
    data = response.json()
    # user_id and created_at should be present
    assert isinstance(data["user_id"], int)
    assert isinstance(data["created_at"], s

### Challenge 2 (Intermediate): Generating Edge Case Tests

**Task:** Prompt the LLM to generate tests for common edge cases, such as providing invalid data or requesting a non-existent resource.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to write two new test functions:
    * A test for the `POST /users/` endpoint that tries to create a user with an email that already exists, asserting that the API returns a `400 Bad Request` error.
    * A test for the `GET /users/{user_id}` endpoint that requests a non-existent user ID, asserting that the API returns a `404 Not Found` error.

**Expected Quality:** Two new `pytest` functions that verify the application handles common error scenarios correctly.

In [ ]:
# Prompt for LLM to generate edge case tests for the API
edge_case_tests_prompt = f"""
You are a QA Engineer. Given the following FastAPI application code, write two pytest test functions to verify the API handles common error scenarios correctly.

Requirements:
- Write a test for the POST /users/ endpoint that attempts to create a user with an email that already exists. Assert that the API returns a 400 Bad Request error.
- Write a test for the GET /users/{{user_id}} endpoint that requests a non-existent user ID. Assert that the API returns a 404 Not Found error.
- Use pytest conventions and best practices.
- The tests should be self-contained and runnable as a script.
- Focus only on these edge cases; do not include happy path tests.

Application code context:
""" + app_code + """

(no markdown fences)
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Edge Case Tests ---
import pytest
from fastapi.testclient import TestClient
from datetime import datetime
from main import app, SessionLocal, Base, engine, UserDB

@pytest.fixture(autouse=True)
def setup_and_teardown_db():
    # Ensure a clean database for each test
    Base.metadata.drop_all(bind=engine)
    Base.metadata.create_all(bind=engine)
    yield
    Base.metadata.drop_all(bind=engine)

@pytest.fixture
def client():
    return TestClient(app)

def test_create_user_with_duplicate_email_returns_409(client):
    user_data = {
        "first_name": "Alice",
        "last_name": "Smith",
        "email": "alice@example.com",
        "password_hash": "hashedpw1",
        "role_id": 1,
        "manager_id": None,
        "team_id": None,
        "start_date": "2023-01-01T09:00:00",
        "gamification_points": 0,
        "profile_picture_url": None
    }
    # Create the user the first time
    response = client.post("/users/", json=user_data)
    assert response.st

### Challenge 3 (Advanced): Testing with an Isolated Database Fixture

**Task:** Generate a `pytest` fixture that creates a fresh, isolated, in-memory database for each test session. Then, refactor your tests to use this fixture. This is a critical pattern for professional-grade testing.

> **Hint:** Why use an isolated database? Running tests against your actual development database can lead to data corruption and flaky, unreliable tests. A pytest fixture that creates a fresh, in-memory database for each test ensures that your tests are independent, repeatable, and have no side effects.

**Instructions:**
1.  Create a prompt that asks the LLM to generate a `pytest` fixture.
2.  This fixture should configure a temporary, in-memory SQLite database using SQLAlchemy.
3.  It needs to create all the database tables before the test runs and tear them down afterward.
4.  Crucially, it must override the `get_db` dependency in your FastAPI app to use this temporary database during tests.
5.  Save the generated fixture code to a special file named `tests/conftest.py`.
6.  Finally, create a new test file, `tests/test_main_with_fixture.py`, and ask the LLM to rewrite the happy-path tests from Challenge 1 to use the new database fixture.

**Expected Quality:** Two new files, `tests/conftest.py` and `tests/test_main_with_fixture.py`, containing a professional `pytest` fixture for database isolation and tests that are correctly refactored to use it.

In [ ]:
# Prompt for LLM to generate the pytest fixture for an isolated test database
db_fixture_prompt = f"""
You are a QA Engineer. Given the following FastAPI application code, write a pytest fixture that creates a fresh, isolated, in-memory SQLite database for each test session using SQLAlchemy.

Requirements:
- The fixture should set up a temporary, in-memory SQLite database.
- It should create all database tables before the tests run and tear them down afterward.
- Override the get_db dependency in the FastAPI app so that all tests use this temporary database.
- Use pytest conventions and best practices.
- Save the fixture code to tests/conftest.py.

Application code context:
""" + app_code + """

(no markdown fences)
"""

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py", overwrite=True)
else:
    print("Skipping fixture generation because app context is missing.")

# Prompt for LLM to refactor happy path tests to use the new fixture
refactor_tests_prompt = f"""
You are a QA Engineer. Given the following FastAPI application code and the pytest fixture for an isolated, in-memory database, rewrite the happy path tests for the API to use the new fixture.

Requirements:
- Refactor the happy path tests for the POST /users/ and GET /users/ endpoints to use the database fixture from tests/conftest.py.
- Ensure the tests are self-contained and runnable as a script.
- Use pytest conventions and best practices.
- Save the refactored tests to tests/test_main_with_fixture.py.

Application code context:
""" + app_code + """

(no markdown fences)
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py", overwrite=True)
else:
    print("Skipping test refactoring because app context is missing.")

--- Generating Pytest DB Fixture ---
# tests/conftest.py

import pytest
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from fastapi.testclient import TestClient

# Import your FastAPI app and SQLAlchemy Base, UserDB, get_db
from main import app, Base, get_db

@pytest.fixture(scope="session")
def db_engine():
    """
    Create a fresh in-memory SQLite engine for the test session.
    """
    engine = create_engine(
        "sqlite:///:memory:",
        connect_args={"check_same_thread": False}
    )
    Base.metadata.create_all(bind=engine)
    yield engine
    Base.metadata.drop_all(bind=engine)
    engine.dispose()

@pytest.fixture(scope="session")
def db_session_factory(db_engine):
    """
    Returns a sessionmaker, bound to the test engine.
    """
    return sessionmaker(autocommit=False, autoflush=False, bind=db_engine)

@pytest.fixture(scope="function")
def db_session(db_session_factory):
    """
    Yields a new database session for a test functio

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality.

> **Key Takeaway:** Using AI to generate tests is a massive force multiplier for quality assurance. It excels at creating boilerplate test code, brainstorming edge cases, and generating complex setup fixtures, allowing developers to build more reliable software faster.